In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import os
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)


2.6.0


In [2]:
# Set parameter
nb_batch = 256
num_classes = 10
nb_epochs = 200
data_augmentation = True
num_predictions = 20


In [3]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print('x_train dtype:', x_train.dtype)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


x_train shape: (50000, 32, 32, 3)
x_train dtype: uint8
50000 train samples
10000 test samples


In [4]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']


In [5]:
# 전처리
AUTOTUNE = tf.data.AUTOTUNE
normalization_layer = keras.layers.experimental.preprocessing.Rescaling(1./255)
data_augmentation_layer = Sequential([
    keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    keras.layers.experimental.preprocessing.RandomRotation(0.1)
])


def prepare(ds, shuffle=False, augment=False):
    # Resize and rescale all datasets
    ds = ds.map(lambda x, y: (normalization_layer(x), y),
                num_parallel_calls=AUTOTUNE)

    if shuffle:
        ds = ds.shuffle(1000)

    # Batch all datasets
    ds = ds.batch(nb_batch)

    # Use data augmentation only on the training set
    if augment:
        ds = ds.map(lambda x, y: (data_augmentation_layer(x, training=True), y),
                    num_parallel_calls=AUTOTUNE)

    # Use buffered prefecting on all datasets
    return ds.prefetch(buffer_size=AUTOTUNE)


In [6]:
# 전처리
y_train = y_train.reshape(-1)
y_test = y_test.reshape(-1)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_ds = prepare(train_ds, shuffle=True, augment=True)
test_ds = prepare(test_ds)


In [7]:
def getModel(dropout_rate=0.0):
    model = Sequential()
    model.add(Conv2D(32, [3, 3], padding='same',
              activation='relu', input_shape=x_train.shape[1:]))
    model.add(keras.layers.BatchNormalization())
    model.add(Conv2D(32, [3, 3], padding='same', activation='relu'))
    model.add(keras.layers.BatchNormalization())
    model.add(Conv2D(32, [3, 3], padding='same', activation='relu'))
    model.add(keras.layers.BatchNormalization())
    model.add(MaxPooling2D([2, 2], strides=(2, 2), padding='same'))
    model.add(Dropout(dropout_rate))

    model.add(Conv2D(64, [3, 3], padding='same', activation='relu'))
    model.add(keras.layers.BatchNormalization())
    model.add(Conv2D(64, [3, 3], padding='same', activation='relu'))
    model.add(keras.layers.BatchNormalization())
    model.add(Conv2D(64, [3, 3], padding='same', activation='relu'))
    model.add(keras.layers.BatchNormalization())
    model.add(MaxPooling2D([2, 2], strides=(2, 2), padding='same'))
    model.add(Dropout(dropout_rate))

    model.add(Conv2D(128, [3, 3], padding='same', activation='relu'))
    model.add(keras.layers.BatchNormalization())
    model.add(Conv2D(128, [3, 3], padding='same', activation='relu'))
    model.add(keras.layers.BatchNormalization())
    model.add(Conv2D(128, [3, 3], padding='same', activation='relu'))
    model.add(keras.layers.BatchNormalization())
    model.add(MaxPooling2D([2, 2], strides=(2, 2), padding='same'))
    model.add(Dropout(dropout_rate))

    model.add(Conv2D(256, [3, 3], padding='same', activation='relu'))
    model.add(keras.layers.BatchNormalization())
    model.add(Conv2D(256, [3, 3], padding='same', activation='relu'))
    model.add(keras.layers.BatchNormalization())
    model.add(Conv2D(256, [3, 3], padding='same', activation='relu'))
    model.add(keras.layers.BatchNormalization())
    model.add(MaxPooling2D([2, 2], strides=(2, 2), padding='same'))
    model.add(Dropout(dropout_rate))

    model.add(Conv2D(512, [3, 3], padding='same', activation='relu'))
    model.add(keras.layers.BatchNormalization())
    model.add(Conv2D(512, [3, 3], padding='same', activation='relu'))
    model.add(keras.layers.BatchNormalization())
    model.add(Conv2D(512, [3, 3], padding='same', activation='relu'))
    model.add(keras.layers.BatchNormalization())
    model.add(MaxPooling2D([2, 2], strides=(2, 2), padding='same'))
    model.add(Dropout(dropout_rate))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(dropout_rate))

    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(dropout_rate))

    model.add(Dense(512, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='softmax'))

    return model


In [8]:
callback = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True,
                                         monitor='val_loss')                                     


In [9]:
history_list = []

for num in range(1, 10):
    model = getModel(num * 0.1)

    model.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
                  optimizer=keras.optimizers.Adam(1e-3),
                  metrics=['accuracy'])

    hist = model.fit(train_ds, batch_size=nb_batch, epochs=nb_epochs,
                     verbose=1, validation_data=test_ds,
                     shuffle=True,
                     callbacks=[callback])

    test_loss, test_acc = model.evaluate(test_ds, verbose=2)
    history_list.append([num*0.1, test_acc * 100])

Epoch 1/200
196/196 [==============================] - 16s 57ms/step - loss: 1.8931 - accuracy: 0.2960 - val_loss: 3.0443 - val_accuracy: 0.1483
Epoch 2/200
196/196 [==============================] - 10s 52ms/step - loss: 1.4500 - accuracy: 0.4674 - val_loss: 2.1218 - val_accuracy: 0.3865
Epoch 3/200
196/196 [==============================] - 10s 52ms/step - loss: 1.2243 - accuracy: 0.5679 - val_loss: 1.4595 - val_accuracy: 0.5777
Epoch 4/200
196/196 [==============================] - 10s 53ms/step - loss: 1.0676 - accuracy: 0.6292 - val_loss: 1.5313 - val_accuracy: 0.5920
Epoch 5/200
196/196 [==============================] - 11s 53ms/step - loss: 0.9588 - accuracy: 0.6722 - val_loss: 1.0502 - val_accuracy: 0.6820
Epoch 6/200
196/196 [==============================] - 11s 53ms/step - loss: 0.8799 - accuracy: 0.7025 - val_loss: 0.9962 - val_accuracy: 0.6954
Epoch 7/200
196/196 [==============================] - 11s 54ms/step - loss: 0.8231 - accuracy: 0.7253 - val_loss: 0.9421 - val_ac

In [11]:
for hist in history_list:
    print('\nDropout rate: {:0.1f}, Test accuracy: {:.2f}%'.format(
        hist[0], hist[1]))



Dropout rate: 0.1, Test accuracy: 86.87%

Dropout rate: 0.2, Test accuracy: 86.88%

Dropout rate: 0.3, Test accuracy: 87.61%

Dropout rate: 0.4, Test accuracy: 84.98%

Dropout rate: 0.5, Test accuracy: 80.63%

Dropout rate: 0.6, Test accuracy: 67.46%

Dropout rate: 0.7, Test accuracy: 55.39%

Dropout rate: 0.8, Test accuracy: 10.00%

Dropout rate: 0.9, Test accuracy: 10.00%
